# Download the scRNA data

In [25]:
data_dir = '/root/datos/maestria/netopaas/'
backup_dir = '/root/datos/maestria/netopaas/luca_explore/surgeries'

In [27]:
import numpy as np
import pandas as pd
import utils.functions as ut
import urllib

import scanpy as sc

import rpy2.rinterface_lib.callbacks
import logging
from rpy2.robjects import pandas2ri
import rpy2.robjects as robjects

# Set the graphics device
robjects.r('options(bitmapType="cairo")')
import anndata2ri

# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

/tmp/ipykernel_33074/1855495475.py:23: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


## Zuani 2024

### Download

In [28]:
zuani_dir = f'{data_dir}/Zuani2024'
zuani_samples = pd.read_csv(f'{zuani_dir}/E-MTAB-13526.sdrf.txt', sep='\t')

# Filter for non enriched MDSC = Myeloid derived supressor cells, CD235a- filters for erythrocites
zuani_samples

,Source Name,Comment[ENA_SAMPLE],Comment[BioSD_SAMPLE],Characteristics[organism],Characteristics[individual],Characteristics[original source name],Characteristics[age],Unit[time unit],Term Source REF,Term Accession Number,...,Derived Array Data File,Protocol REF.6,Protocol REF.7,Derived Array Data File.1,Protocol REF.8,Protocol REF.9,Derived Array Data File.2,Factor Value[disease],Factor Value[FACS],Factor Value[sampling site]
0,P1_B1,ERS16703054,SAMEA114591031,Homo sapiens,Patient 1,TB18.0655,73,year,EFO,UO_0000036,...,P1_B1-matrix.mtx.gz,P-MTAB-137970,P-MTAB-137971,P1_B1-barcodes.tsv.gz,P-MTAB-137970,P-MTAB-137971,P1_B1-features.tsv.gz,non-small cell lung cancer,MDSC,normal tissue adjacent to tumor
1,P1_B1,ERS16703054,SAMEA114591031,Homo sapiens,Patient 1,TB18.0655,73,year,EFO,UO_0000036,...,P1_B1-matrix.mtx.gz,P-MTAB-137970,P-MTAB-137971,P1_B1-barcodes.tsv.gz,P-MTAB-137970,P-MTAB-137971,P1_B1-features.tsv.gz,non-small cell lung cancer,MDSC,normal tissue adjacent to tumor
2,P1_B1,ERS16703054,SAMEA114591031,Homo sapiens,Patient 1,TB18.0655,73,year,EFO,UO_0000036,...,P1_B1-matrix.mtx.gz,P-MTAB-137970,P-MTAB-137971,P1_B1-barcodes.tsv.gz,P-MTAB-137970,P-MTAB-137971,P1_B1-features.tsv.gz,non-small cell lung cancer,MDSC,normal tissue adjacent to tumor
3,P1_B1,ERS16703054,SAMEA114591031,Homo sapiens,Patient 1,TB18.0655,73,year,EFO,UO_0000036,...,P1_B1-matrix.mtx.gz,P-MTAB-137970,P-MTAB-137971,P1_B1-barcodes.tsv.gz,P-MTAB-137970,P-MTAB-137971,P1_B1-features.tsv.gz,non-small cell lung cancer,MDSC,normal tissue adjacent to tumor
4,P1_B1,ERS16703054,SAMEA114591031,Homo sapiens,Patient 1,TB18.0655,73,year,EFO,UO_0000036,...,P1_B1-matrix.mtx.gz,P-MTAB-137970,P-MTAB-137971,P1_B1-barcodes.tsv.gz,P-MTAB-137970,P-MTAB-137971,P1_B1-features.tsv.gz,non-small cell lung cancer,MDSC,normal tissue adjacent to tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1562,P18_T1,ERS16703118,SAMEA114591092,Homo sapiens,Patient 18,TB21.0006,77,year,EFO,UO_0000036,...,P18_T1-matrix.mtx.gz,P-MTAB-137970,P-MTAB-137971,P18_T1-barcodes.tsv.gz,P-MTAB-137970,P-MTAB-137971,P18_T1-features.tsv.gz,lung squamous cell carcinoma,CD45+,tumor
1563,P18_T1,ERS16703118,SAMEA114591092,Homo sapiens,Patient 18,TB21.0006,77,year,EFO,UO_0000036,...,P18_T1-matrix.mtx.gz,P-MTAB-137970,P-MTAB-137971,P18_T1-barcodes.tsv.gz,P-MTAB-137970,P-MTAB-137971,P18_T1-features.tsv.gz,lung squamous cell carcinoma,CD45+,tumor
1564,P18_T1,ERS16703118,SAMEA114591092,Homo sapiens,Patient 18,TB21.0006,77,year,EFO,UO_0000036,...,P18_T1-matrix.mtx.gz,P-MTAB-137970,P-MTAB-137971,P18_T1-barcodes.tsv.gz,P-MTAB-137970,P-MTAB-137971,P18_T1-features.tsv.gz,lung squamous cell carcinoma,CD45+,tumor
1565,P18_T1,ERS16703118,SAMEA114591092,Homo sapiens,Patient 18,TB21.0006,77,year,EFO,UO_0000036,...,P18_T1-matrix.mtx.gz,P-MTAB-137970,P-MTAB-137971,P18_T1-barcodes.tsv.gz,P-MTAB-137970,P-MTAB-137971,P18_T1-features.tsv.gz,lung squamous cell carcinoma,CD45+,tumor


We filter for non-immune enriched samples and only tumor tissue. All the non-tumor enirhced ones are filtered for no plasma cells (erythrocites CD235a-)
that is also the case in many of the other samples... we can consider the eryhrocites enriched depleted, for they are only one cell type that is not very present. But we should impute some abundance values in these samples.

In [5]:
facs_filter = ['CD235a-']
zuani_samples = zuani_samples[zuani_samples['Factor Value[FACS]'].isin(facs_filter)]

zuani_samples = zuani_samples[zuani_samples['Factor Value[sampling site]']=='tumor']

In [7]:
zuani_samples.columns

Index(['Source Name', 'Comment[ENA_SAMPLE]', 'Comment[BioSD_SAMPLE]',
       'Characteristics[organism]', 'Characteristics[individual]',
       'Characteristics[original source name]', 'Characteristics[age]',
       'Unit[time unit]', 'Term Source REF', 'Term Accession Number',
       'Characteristics[developmental stage]', 'Characteristics[sex]',
       'Characteristics[organism part]', 'Characteristics[disease]',
       'Characteristics[FACS]', 'Characteristics[sampling site]',
       'Characteristics[tumor grading]', 'Description', 'Material Type',
       'Protocol REF', 'Protocol REF.1', 'Protocol REF.2', 'Extract Name',
       'Comment[LIBRARY_LAYOUT]', 'Comment[LIBRARY_SELECTION]',
       'Comment[LIBRARY_SOURCE]', 'Comment[LIBRARY_STRATEGY]',
       'Comment[cdna read]', 'Comment[cdna read offset]',
       'Comment[cdna read size]', 'Comment[cell barcode offset]',
       'Comment[cell barcode read]', 'Comment[cell barcode size]',
       'Comment[end bias]', 'Comment[input molecu

In [11]:
zuani_samples['Characteristics[tumor grading]'].unique()

array(['T3N0M0', 'T2bN0M0', nan, 'T2aN0M0', 'T4N2M0', 'T2aN1M0', 'T3M0N0',
       'T2bN1M0', 'T1cN0M0', 'T4N0'], dtype=object)

In [76]:
base_url = 'https://ftp.ebi.ac.uk/biostudies/fire/E-MTAB-/526/E-MTAB-13526/Files'
zuani_patients = zuani_samples['Source Name'].unique()

urls_paths = []

for pat in zuani_patients:
    filename = f'{pat}-barcodes.tsv.gz'
    urls_paths.append((f'{base_url}/{filename}',f'{zuani_dir}/{filename}'))

    filename = f'{pat}-features.tsv.gz'
    urls_paths.append((f'{base_url}/{filename}',f'{zuani_dir}/{filename}'))

    filename = f'{pat}-matrix.mtx.gz'
    urls_paths.append((f'{base_url}/{filename}',f'{zuani_dir}/{filename}'))

ut.download_parallel(urls_paths, cpus=8)

CPUS:  8
getting https://ftp.ebi.ac.uk/biostudies/fire/E-MTAB-/526/E-MTAB-13526/Files/P1_T1-barcodes.tsv.gz
getting https://ftp.ebi.ac.uk/biostudies/fire/E-MTAB-/526/E-MTAB-13526/Files/P1_T1-features.tsv.gz
getting https://ftp.ebi.ac.uk/biostudies/fire/E-MTAB-/526/E-MTAB-13526/Files/P1_T1-matrix.mtx.gz
getting https://ftp.ebi.ac.uk/biostudies/fire/E-MTAB-/526/E-MTAB-13526/Files/P2_T1-barcodes.tsv.gz
getting https://ftp.ebi.ac.uk/biostudies/fire/E-MTAB-/526/E-MTAB-13526/Files/P2_T1-features.tsv.gz
getting https://ftp.ebi.ac.uk/biostudies/fire/E-MTAB-/526/E-MTAB-13526/Files/P2_T1-matrix.mtx.gz
getting https://ftp.ebi.ac.uk/biostudies/fire/E-MTAB-/526/E-MTAB-13526/Files/P15_T2-barcodes.tsv.gz
getting https://ftp.ebi.ac.uk/biostudies/fire/E-MTAB-/526/E-MTAB-13526/Files/P15_T2-features.tsv.gz
url: https://ftp.ebi.ac.uk/biostudies/fire/E-MTAB-/526/E-MTAB-13526/Files/P2_T1-features.tsv.gz time (s): 1.5921189785003662
getting https://ftp.ebi.ac.uk/biostudies/fire/E-MTAB-/526/E-MTAB-13526/Files

### Remove empty drops

In [23]:
from os import system as sys
import os

codeDir = '/root/host_home/luca/utils/' # CHANGE ME TO THE DIRECTORY WHERE run_EmptyDrop.R IS CLONED
directory = '/root/datos/maestria/netopaas/Zuani2024/' # change me to the dir where the matrix files are stored

# All CellRanger samples for the lung data
samples = []

file_pattern = sorted([f for f in os.listdir(directory) if 'matrix' in f])
for mtx_file in file_pattern:
    idx = mtx_file.replace('matrix.mtx.gz', '')
    samples.append(idx)

for sample in samples:
    fname = directory+"/EmptyDrop_"+sample+".sh"
    f = open(f'{fname}', 'w')
    f.write("cd /root/datos/maestria/netopaas/Zuani2024/\n") # CHANGE ME TO A PLACE YOU WOULD LIKE TO WRITE OUTPUTS TO
    # f.write("conda activate minimal_env\n" )
    f.write("Rscript " + codeDir + "run_EmptyDrop.R {0}\n".format(str(sample)))
    f.close()

    # Example batch calling on the codon LSF cluster (this will vary depending on the system, bsub vs. qsub etc)
    print(f'bash {fname}')
    sys(f'bash {fname}')

bash /root/datos/maestria/netopaas/Zuani2024//EmptyDrop_P16_T2-.sh
[1] "/root/datos/maestria/netopaas/Zuani2024//outputEmptyDrops/P16_T2-"
[1] "Creating folder  /root/datos/maestria/netopaas/Zuani2024//outputEmptyDrops/P16_T2-"


Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOr

bash /root/datos/maestria/netopaas/Zuani2024//EmptyDrop_P17_T2-.sh
[1] "/root/datos/maestria/netopaas/Zuani2024//outputEmptyDrops/P17_T2-"
[1] "Creating folder  /root/datos/maestria/netopaas/Zuani2024//outputEmptyDrops/P17_T2-"


Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOr

bash /root/datos/maestria/netopaas/Zuani2024//EmptyDrop_P17_T3-.sh
[1] "/root/datos/maestria/netopaas/Zuani2024//outputEmptyDrops/P17_T3-"
[1] "Creating folder  /root/datos/maestria/netopaas/Zuani2024//outputEmptyDrops/P17_T3-"


...

Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOr

bash /root/datos/maestria/netopaas/Zuani2024//EmptyDrop_P4_T3-.sh
[1] "/root/datos/maestria/netopaas/Zuani2024//outputEmptyDrops/P4_T3-"
[1] "Creating folder  /root/datos/maestria/netopaas/Zuani2024//outputEmptyDrops/P4_T3-"


Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOr

bash /root/datos/maestria/netopaas/Zuani2024//EmptyDrop_P8_T2-.sh
[1] "/root/datos/maestria/netopaas/Zuani2024//outputEmptyDrops/P8_T2-"
[1] "Creating folder  /root/datos/maestria/netopaas/Zuani2024//outputEmptyDrops/P8_T2-"


Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOr

### Convert to adata

In [33]:
import os
import io
import csv
import gzip

import scanpy as sc
import anndata as ad

def concatenate_datasets(directory):
    """
    Concatenate multiple .mtx datasets from a single directory into a single AnnData object.
    
    Parameters:
    - directory (str): The directory containing .mtx, features.tsv, and barcodes.tsv files.
    
    Returns:
    - AnnData: The concatenated AnnData object.
    """
    adatas = {}
    features_file = ''
    
    # List all subdirectories in the main directory
    subdirectories = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
    
    for subdirectory in subdirectories:
        # Construct the path to the subdirectory
        subdirectory_path = os.path.join(directory, subdirectory)
        
        # Read the .mtx file and associated files from the subdirectory
        adata = sc.read_10x_mtx(subdirectory_path, var_names='gene_symbols', cache=True)
        
        # Use the subdirectory name as the identifier
        idx = subdirectory
        
        # Store the AnnData object in the dictionary
        adatas[idx] = adata
    
    # Concatenate all AnnData objects, labeling them by the subdirectory name which is used as the sample identifier
    concatenated_adata = ad.concat(adatas, label='sample')

    return concatenated_adata

# Example usage:
adata = concatenate_datasets(f'{zuani_dir}/outputEmptyDrops')
print(adata)

/usr/local/lib/python3.11/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/usr/local

AnnData object with n_obs × n_vars = 318313 × 33538
    obs: 'sample'


/usr/local/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [36]:
adata.write_h5ad(f'{zuani_dir}/Zuani2024.h5ad')

In [63]:
adata = sc.read_h5ad('/root/datos/maestria/netopaas/Zuani2024/Zuani.h5ad')

/usr/local/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Before emptydrops with FDR 0.001

In [5]:
adata

AnnData object with n_obs × n_vars = 101923200 × 33538
    obs: 'sample'
    var: 'gene_symbols'

adter EmptyDrops

In [116]:
adata

AnnData object with n_obs × n_vars = 312502 × 33538
    obs: 'sample', 'barcode', 'Source Name', 'Characteristics[tumor grading]'

In [94]:
adata.obs['sample'] = adata.obs['sample'].str[:-1]
zuani_samples_unique = zuani_samples.drop_duplicates(subset=['Source Name'], keep='first')
adata.obs = pd.merge(adata.obs, zuani_samples_unique.loc[:, ['Source Name', 'Characteristics[tumor grading]']],
                            left_on='sample', right_on='Source Name', how='left')

/usr/local/lib/python3.11/site-packages/anndata/_core/anndata.py:850: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


In [100]:
import re
def map_tnm_to_roman(tnm_stage):
    stage_mappings = {
        r"^Tis N0 M0$": "Stage 0",
        r"^T1[ab]? N0 M0$": "Stage I",
        r"^T2[ab]? N0 M0$": "Stage II",
        r"^T2[ab]? N0/1$": "Stage II",
        r"^T1[ab]? N1 M0$": "Stage II",
        r"^T2[ab]? N1 M0$": "Stage II",
        r"^T3 N[01] M0$": "Stage II",
        r"^T1[ab]? N2 M0$": "Stage III",
        r"^T2[ab]? N2 M0$": "Stage III",
        r"^T3 N2 M0$": "Stage III",
        r"^T4 N[012] M0$": "Stage III",
        r"^T4 N0$": "Stage III",
        r"^T[1-4][ab]? N3 M0$": "Stage III",
        r"^T[1-4][ab]? N[0-3] M1[abc]$": "Stage IV",
    }
    
    for pattern, stage in stage_mappings.items():
        if re.match(pattern, tnm_stage):
            return stage
    return "Unknown stage"

Remove the patients with no tumor grading

In [111]:
adata = adata[~adata.obs['Characteristics[tumor grading]'].isna()]

In [ ]:
adata.obs['Characteristics[tumor grading]'].apply(map_tnm_to_roman)
# adata.obs.stage

In [115]:
adata.write_h5ad(f'{zuani_dir}/Zuani2024.h5ad')

/usr/local/lib/python3.11/site-packages/anndata/_core/anndata.py:1292: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/usr/local/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.11/site-packages/anndata/_core/anndata.py:1292: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/usr/local/lib/python3.11/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/lo

## Deng 2024

In [1]:
# urllib.request.urlretrieve('https://figshare.com/ndownloader/files/44695465',f'{data_dir}/Deng2024/dengluad.rds')
urllib.request.urlretrieve('https://figshare.com/ndownloader/files/46617667',f'{data_dir}/Deng2024/dengEcoTyper.rds')


NameError: name 'urllib' is not defined

In [3]:
! rm /root/datos/maestria/netopaas/Deng2024/dengRNA.h5ad

In [8]:
%%R -i data_dir
library(SeuratDisk)
Convert(paste0(data_dir, '/Deng2024/deng.h5Seurat'), dest = paste0(data_dir, '/Deng2024/dengRNA.h5ad'), assay = 'RNA')


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat
Validating h5Seurat file
Adding data from RNA as X
Adding counts from RNA as raw
Transfering meta.data to obs
Adding dimensional reduction information for tsne (global)
Adding dimensional reduction information for umap (global)


In [3]:
%%R -i data_dir
library(Seurat)
library(SeuratDisk)

deng <- readRDS(paste0(data_dir, '/Deng2024/dengEcoTyper.rds'))
deng = UpdateSeuratObject(deng)

# After P49 there is no metadata so better delete. Also the ones with N as they are healthy controls,
cells_to_exclude <- which(grepl("P05|N", deng@meta.data$orig.ident))
deng <- subset(deng, cells = setdiff(Cells(deng), cells_to_exclude))


SaveH5Seurat(deng, filename=paste0(data_dir, '/Deng2024/deng.h5Seurat'))
# Convert(paste0(data_dir, '/Deng2024/deng.h5Seurat'), dest = "h5ad")
Convert(paste0(data_dir, '/Deng2024/deng.h5Seurat'), dest = paste0(data_dir, '/Deng2024/dengRNA.h5ad'), assay = 'RNA')


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    Loading required package: SeuratObject
Loading required package: sp

Attaching package: ‘SeuratObject’

The following object is masked from ‘package:base’:

    intersect

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat
Validating object structure
Updating object slots
Ensuring keys are in the proper structure
Updating matrix keys for DimReduc ‘pca’
Updating matrix keys for DimReduc ‘umap’
Updating matrix keys for DimReduc ‘tsne’
Ensuring keys are in the proper structure
Ensuring feature names don't have underscores or pipes
Updating slots in RNA
Updating slots in integrated
Updating slots in integrated_nn
Setting default assay of integrated_nn to integrated
Updating slots in integrated_snn
Setting default assay of integrated_snn to integra

RInterpreterError: Failed to parse and evaluate line 'library(Seurat)\nlibrary(SeuratDisk)\n\ndeng <- readRDS(paste0(data_dir, \'/Deng2024/dengEcoTyper.rds\'))\ndeng = UpdateSeuratObject(deng)\n\n# After P49 there is no metadata so better delete. Also the ones with N as they are healthy controls,\ncells_to_exclude <- which(grepl("P05|N", deng@meta.data$orig.ident))\ndeng <- subset(deng, cells = setdiff(Cells(deng), cells_to_exclude))\n\n\nSaveH5Seurat(deng, filename=paste0(data_dir, \'/Deng2024/deng.h5Seurat\'))\n# Convert(paste0(data_dir, \'/Deng2024/deng.h5Seurat\'), dest = "h5ad")\nConvert(paste0(data_dir, \'/Deng2024/deng.h5Seurat\'), dest = paste0(data_dir, \'/Deng2024/dengRNA.h5ad\'), assay = \'RNA\')\n'.
R error message: 'Error: Destination H5AD file exists'
R stdout:
Loading required package: SeuratObject
Loading required package: sp

Attaching package: ‘SeuratObject’

The following object is masked from ‘package:base’:

    intersect

Registered S3 method overwritten by 'Seura

In [14]:
# adata_integ = sc.read_h5ad(f'{data_dir}/Deng2024/deng.h5ad')
adata = sc.read_h5ad(f'{data_dir}/Deng2024/dengRNA.h5ad')

# After P49 there is no metadata so better delete. Also the ones with N as they are healthy controls,
# Should be done before in R to be faster
# adata = adata[~adata.obs['orig.ident'].str.contains('P05|N')]

# The assignment fo slots and assays is not very good with Convert from SeuratDisk so we make some arrangements
# We also arrange the X so that it is like the other filtered datasets in the repo
adata.layers['data'] = adata.X.copy()
adata.X = adata.raw.X

adata.raw = adata_integ

path_metadata = f'{data_dir}/Deng2024/metadata.xlsx'
if not os.path.exists(path_metadata):
    urllib.request.urlretrieve('https://www.ncbi.nlm.nih.gov/pmc/articles/PMC11031428/bin/mmc2.xlsx', path_metadata)
xl_zuani = pd.ExcelFile(path_metadata)
metada_zuani = xl_zuani.parse(' lung cancer in scRNAseq')
adata.obs = pd.merge(adata.obs, metada_zuani, how='left', left_on='orig.ident', right_on='Patient Number')


adata.write_h5ad(f'{backup_dir}/filtered_Deng_Liu_LUAD_2024.h5ad')

NameError: name 'adata_integ' is not defined

In [21]:
adata.obs.ecoRecovery.unique()

array(['unassigned', 'CD8.T.cells_S04', 'CD4.T.cells_S04',
       'CD4.T.cells_S05', 'Mast.cells_S01', 'CD8.T.cells_S03',
       'NK.cells_S04', 'CD4.T.cells_S03', 'CD8.T.cells_S01',
       'B.cells_S02', 'CD4.T.cells_S01', 'Monocytes.and.Macrophages_S05',
       'NK.cells_S02', 'NK.cells_S01', 'CD4.T.cells_S02',
       'CD8.T.cells_S02', 'NK.cells_S03', 'Dendritic.cells_S01',
       'PMNs_S01', 'Monocytes.and.Macrophages_S04', 'B.cells_S01',
       'Monocytes.and.Macrophages_S02', 'B.cells_S03',
       'Monocytes.and.Macrophages_S03', 'PMNs_S03', 'Dendritic.cells_S02',
       'Mast.cells_S02', 'Monocytes.and.Macrophages_S01',
       'Epithelial.cells_S02', 'PMNs_S02', 'Fibroblasts_S03',
       'Endothelial.cells_S02', 'Epithelial.cells_S03', 'Fibroblasts_S01',
       'Endothelial.cells_S01', 'Endothelial.cells_S04',
       'Fibroblasts_S04', 'Epithelial.cells_S01', 'Endothelial.cells_S05',
       'Mast.cells_S03', 'Mast.cells_S04', 'Fibroblasts_S02',
       'Endothelial.cells_S03'], d

In [17]:
adata

AnnData object with n_obs × n_vars = 488236 × 33336
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'assigned_cell_type', 'stage', 'Ecotype', 'ecoRecovery'
    var: 'features'
    obsm: 'X_tsne', 'X_umap'
    layers: 'data'

In [277]:
! {data_dir}/Deng2024/dengRNA.h5ad

244372.54s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
/usr/bin/sh: 1: /root/datos/maestria/netopaas//Deng2024/dengRNA.h5ad: Permission denied


In [278]:
! {data_dir}/Deng2024/deng.h5ad

244384.51s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
/usr/bin/sh: 1: /root/datos/maestria/netopaas//Deng2024/.h5ad: not found


## Alexandra 2023

In [77]:
urllib.request.urlretrieve('https://zenodo.org/records/7852154/files/data_code.zip?download=1',f'{data_dir}/Alexandra2023/data.zip')

('/root/datos/maestria/netopaas//alexandra2023/data.zip',
 <http.client.HTTPMessage at 0x7f7325a5cf10>)

## Zhong 2024


In [79]:
zhong_samples = pd.read_csv(f'{data_dir}/Zhong2024/GSE241934_IIT_Meta.txt', sep='\t')
zhong_samples

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,sampleID,cellID,RNA_snn_res.0.8,seurat_clusters,major.cell.type,RNA_snn_res.1,...,Gender,Age,Histology,Cycles,PD1,Pathological Response,EGFR,Pathological Response Rate,Smoking_History,PD-L1 TPS
0,P343,17606,4574,2.891060,P343,P343_AAACCTGAGCTATGCT-1,6,6,Fibro,NaN,...,F,60,LUAD,3,Sintilimab,non-MPR,L858R,0.85,N,0.01
1,P343,9542,3230,4.003354,P343,P343_AAACCTGCATGATCCA-1,8,8,Fibro,NaN,...,F,60,LUAD,3,Sintilimab,non-MPR,L858R,0.85,N,0.01
2,P343,10729,3832,2.302172,P343,P343_AAACCTGGTGAGTATA-1,10,10,Endo,NaN,...,F,60,LUAD,3,Sintilimab,non-MPR,L858R,0.85,N,0.01
3,P343,7788,2859,5.225989,P343,P343_AAACCTGTCCAGTATG-1,10,10,Endo,NaN,...,F,60,LUAD,3,Sintilimab,non-MPR,L858R,0.85,N,0.01
4,P343,15449,4074,3.301185,P343,P343_AAACGGGCACGACGAA-1,10,10,Endo,NaN,...,F,60,LUAD,3,Sintilimab,non-MPR,L858R,0.85,N,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78686,P591,6608,2528,3.344431,P591,P591_TGCCCTAGTCAGATAA-1,7,7,Epi,NaN,...,M,51,LUAD,3,Sintilimab,non-MPR,KDD,0.58,Y,<1%
78687,P591,1064,707,7.518797,P591,P591_TGGCTGGAGCGTTGCC-1,7,7,Epi,NaN,...,M,51,LUAD,3,Sintilimab,non-MPR,KDD,0.58,Y,<1%
78688,P591,13226,3318,1.126569,P591,P591_TTCTCCTAGTGCGTGA-1,7,7,Epi,NaN,...,M,51,LUAD,3,Sintilimab,non-MPR,KDD,0.58,Y,<1%
78689,P591,7335,2912,6.693933,P591,P591_TTTCCTCAGCGTAATA-1,7,7,Epi,NaN,...,M,51,LUAD,3,Sintilimab,non-MPR,KDD,0.58,Y,<1%
